<a href="https://colab.research.google.com/github/dylanesq/2-Months-Project-LLM/blob/main/LlamaIndexV3_InbuiltEval_Qwen2_0_5B_Instruct.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install llama-index

#Loading

 **SIMPLE DIRECTORY READER**
READING PDFS

In [ ]:
from llama_index.core import SimpleDirectoryReader

reader=SimpleDirectoryReader(input_dir="/content/", required_exts=[".pdf"])
document_1=reader.load_data()

# Chunking Randomly
[TokenTextSplitter](https://docs.llamaindex.ai/en/v0.10.17/api/llama_index.core.node_parser.TokenTextSplitter.html#tokentextsplitter)

In [ ]:
from llama_index.core.node_parser import TokenTextSplitter
splitter = TokenTextSplitter(
    chunk_size=1024,
    chunk_overlap=100,
    separator=" ",
)

*For Review Purpose*

In [ ]:
token_nodes = splitter.get_nodes_from_documents(
    document_1, show_progress=True
)

Parsing nodes:   0%|          | 0/15 [00:00<?, ?it/s]

# LLM Setup
*   [HuggingFaceLLM](https://docs.llamaindex.ai/en/v0.9.48/api_reference/llms/huggingface.html#huggingfacellm)
*   [Llama 3.2 3B
](https://huggingface.co/meta-llama/Llama-3.2-3B-Instruct)



update : logging without CLI *(contains exposed HF Token , how to fix)*

#LLM Init.

In [ ]:
!pip install llama_index.llms.huggingface
from llama_index.llms.huggingface import HuggingFaceLLM

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 111.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 86.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 78.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

system prompt

In [ ]:
system_prompt="""
You are a Q&A assistant. Your goal is to answer questions as
accurately as possible based on the instructions and context provided.
"""


Quantization [ 8 bit Working]

[HF Bits&Bytesconfig](https://huggingface.co/docs/transformers/en/main_classes/quantization#transformers.BitsAndBytesConfig)
[Bitnbytes docu](https://pypi.org/project/bitsandbytes/https://)

[Why Quantize](https://medium.com/@techresearchspace/what-is-quantization-in-llm-01ba61968a51)

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = "Qwen/Qwen2-0.5B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id,torch_dtype="auto")




/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

# **Embedding & Vectorization**


Vector Imports

Embedding Imports
* [HuggingFaceEmbeddings](https://docs.llamaindex.ai/en/stable/examples/embeddings/huggingface/)  
*   [LAngchain HF Embedding](https://python.langchain.com/api_reference/huggingface/embeddings/langchain_huggingface.embeddings.huggingface.HuggingFaceEmbeddings.html#huggingfaceembeddings)
*   [SBERT](https://www.sbert.net/)
*   [Llamindex embedding](https://docs.llamaindex.ai/en/stable/api_reference/embeddings/huggingface/)





In [ ]:
!pip install llama_index.embeddings.huggingface
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
embed_model  = HuggingFaceEmbedding(model_name = "sentence-transformers/all-MiniLM-L6-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Wrapper [not working without it]

In [ ]:
llm = HuggingFaceLLM(
    #context_window=1024,
    max_new_tokens=512,
    generate_kwargs={"temperature": 0.0, "do_sample": False},
    system_prompt=system_prompt,
    model=model,
    tokenizer=tokenizer,
    tokenizer_kwargs={"max_length": 512},
)

# Service Context/Settings

Set Up
Edit  : ServiceContext is deprecated.

*   Use llama_index.settings.Settings instead
*   [Migration to Settings Llama Index](https://docs.llamaindex.ai/en/stable/module_guides/supporting_modules/service_context_migration/)


In [ ]:
from llama_index.core import Settings
Settings.llm=llm
Settings.embed_model=embed_model
Settings.node_parser=splitter
Settings.chunk_size = 1024

#Indexing

In [ ]:
from llama_index.core import VectorStoreIndex
index=VectorStoreIndex.from_documents(document_1)#no longer required to have servicecontext
#query_engine=index.as_query_engine(topK=5)

#Query Engine



*   [Querying LlamaIndex](https://docs.llamaindex.ai/en/stable/module_guides/querying/)
*   [Query Engine Module
](https://docs.llamaindex.ai/en/stable/module_guides/deploying/query_engine/modules/)


Custom Query engine **[working]**

In [ ]:
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine

#retriever=VectorIndexRetriever(index=index,similarity_top_k=5)
query_engine = index.as_query_engine(topK=5)
#query_engine=RetrieverQueryEngine(retriever=retriever)


**Vanilla Query Response**

In [ ]:
prompt="Omelette recipe ?"

Responses

In [ ]:
response=query_engine.query(prompt)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [ ]:
print(response)

7. Omelette Recipe
The recipe for an omelette includes mixing eggs with salt, pepper, and herbs if desired. It also adds grated cheese and mixed herbs if desired. The omelette is then cooked in a frying pan until it's firm and the cheese begins to melt. After cooking, the omelette is folded in half with a spatula before being served. If using garlic, the recipe calls for adding spring onions, shrimp, chopped onions, and stir-fried vegetables like broccoli or zucchini. The omelette is then topped with bean sprouts, omlette strips, mange tout, coriander, and seasoned seasoning. The omelette is then baked in a high heat for about 30 seconds before being sprinkled with hot sauce.


In [ ]:
retriever=VectorIndexRetriever(index=index,similarity_top_k=5)
norank_retrieved_nodes = retriever.retrieve(prompt)
print(f"Retrieved {len(norank_retrieved_nodes)} nodes:")
for i, node in enumerate(norank_retrieved_nodes):
    print(f"\n--- Node {i + 1} ---")
    print(node.get_content())
    print(f"Score: {node.score}")

Retrieved 5 nodes:

--- Node 1 ---
  
  underneath!
Score: 0.7240149744982657

--- Node 2 ---
  hot.
Score: 0.498612664230735

--- Node 3 ---
  fry
Score: 0.4793515437712278

--- Node 4 ---
  salad
Score: 0.39827164032177426

--- Node 5 ---
  over
Score: 0.3956839454364329


In [ ]:
from llama_index.core.response.pprint_utils import pprint_response
pprint_response(response,show_source=True)
#print(response)

! Insert section for Pre rerank evaluation !


# EVAL
*Types of Metrics*
See Alammar Pg 257
[RAGAS Documentation on the metrics](https://docs.ragas.io/en/stable/concepts/metrics/available_metrics/)

Evaluation Tools

*   [Inbuilt Evaluators](https://docs.llamaindex.ai/en/stable/module_guides/evaluating/usage_pattern/)
*   [DeepEval](https://github.com/confident-ai/deepeval?tab=readme-ov-file#-metrics-and-features)



Inbuilt

In [ ]:
from llama_index.core.evaluation import FaithfulnessEvaluator
evaluator = FaithfulnessEvaluator(llm=llm)
eval_result = evaluator.evaluate_response(response=response)
print(str(eval_result.passing))

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


True


In [ ]:
from llama_index.core.evaluation import RelevancyEvaluator
evaluator2 = RelevancyEvaluator(llm=llm)
eval_result = evaluator.evaluate_response(query=prompt,response=response)
print(str(eval_result.passing))

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


True


In [ ]:
eval_result.contexts

['7. Omelette\t\r \xa0\t\r \xa0\n\t\r \xa0Ingredients\t\r \xa03\t\r \xa0Eggs\t\r \xa0Salt\t\r \xa0and\t\r \xa0Pepper\t\r \xa0Butter\t\r \xa0Grated\t\r \xa0cheese\t\r \xa0(optional)\t\r \xa0Mixed\t\r \xa0herbs\t\r \xa0(optional)\t\r \xa0\t\r \xa0Method\t\r \xa01)\t\r \xa0Mix\t\r \xa0the\t\r \xa0eggs\t\r \xa0in\t\r \xa0any\t\r \xa0sort\t\r \xa0of\t\r \xa0bowl\t\r \xa0or\t\r \xa0jug\t\r \xa0you\t\r \xa0like\t\r \xa0(a\t\r \xa0pint\t\r \xa0glass\t\r \xa0works\t\r \xa0well!)\t\r \xa0with\t\r \xa0salt,\t\r \xa0pepper\t\r \xa0and\t\r \xa0herbs\t\r \xa0if\t\r \xa0desired.\t\r \xa0\t\r \xa02)\t\r \xa0Heat\t\r \xa0the\t\r \xa0butter\t\r \xa0(or\t\r \xa0oil)\t\r \xa0in\t\r \xa0a\t\r \xa0reasonably\t\r \xa0large\t\r \xa0frying\t\r \xa0pan.\t\r \xa0\t\r \xa03)\t\r \xa0Pour\t\r \xa0in\t\r \xa0the\t\r \xa0egg\t\r \xa0mixture\t\r \xa0slowly\t\r \xa0and\t\r \xa0swirl\t\r \xa0the\t\r \xa0pan\t\r \xa0a\t\r \xa0little\t\r \xa0to\t\r \xa0even\t\r \xa0it\t\r \xa0out.\t\r \xa0\t\r \xa04)\t\r \xa0After\t\r \x

In [ ]:
!pip install -U spacy
from llama_index.core.llama_dataset.generator import RagDatasetGenerator
dataset_generator = RagDatasetGenerator.from_documents(
    documents=document_1,
    llm=llm,
    num_questions_per_chunk=1,  # set the number of questions per nodes
)
import torch
torch.device("cuda")
rag_dataset = dataset_generator.generate_questions_from_nodes()
questions = [e.query for e in rag_dataset.examples]

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p'

In [ ]:
questions = [e.query for e in rag_dataset.examples]

In [ ]:
from llama_index.core.evaluation import BatchEvalRunner

runner = BatchEvalRunner(
    {"faithfulness": evaluator, "relevancy": evaluator2},
    workers=8,
)

eval_results = await runner.aevaluate_queries(
    query_engine, queries=questions
)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p'

In [ ]:
print(eval_results.keys())

print(eval_results["faithfulness"][0].dict().keys())

print(eval_results["faithfulness"][0].passing)
print(eval_results["faithfulness"][0].response)
print(eval_results["faithfulness"][0].contexts)

dict_keys(['faithfulness', 'relevancy'])
dict_keys(['query', 'contexts', 'response', 'passing', 'feedback', 'score', 'pairwise_source', 'invalid_result', 'invalid_reason'])
True
1. Chili con carne
2. Beef in beer
3. Pad Thai Chicken
4. Thai Green Curry
5. Pineapple chicken
6. Vegetarian rice
7. Omelette
8. Egg fried rice
9. Salmon in the oven
10. Basic pasta
11. Carbonara al funghi
12. Simple spaghetti bolognaise
13. Cucumber salad
14. Goat cheese and beetroot salad

The recipes involve raw or lightly cooked eggs, which should always use fresh eggs stored according to packaging guidelines.
['INDEX\t\r \xa01. Chili\t\r \xa0con\t\r \xa0carne\t\r \xa02. Beef\t\r \xa0in\t\r \xa0beer\t\r \xa03. Pad\t\r \xa0Thai\t\r \xa0Chicken\t\r \xa04. Thai\t\r \xa0Green\t\r \xa0Curry\t\r \xa05. Pineapple\t\r \xa0chicken\t\r \xa06. Vegetarian\t\r \xa0rice\t\r \xa07. Omelette\t\r \xa08. Egg\t\r \xa0fried\t\r \xa0rice\t\r \xa09. Salmon\t\r \xa0in\t\r \xa0the\t\r \xa0oven\t\r \xa010. Basic\t\r \xa0pasta\t\r 

/tmp/ipython-input-47-1051449714.py:3: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  print(eval_results["faithfulness"][0].dict().keys())


In [ ]:
def get_eval_results(key, eval_results):
    results = eval_results[key]
    correct = 0
    for result in results:
        if result.passing:
            correct += 1
    score = correct / len(results)
    print(f"{key} Score: {score}")
    return score

score1 = get_eval_results("faithfulness", eval_results)

score2 = get_eval_results("relevancy", eval_results)

faithfulness Score: 0.8333333333333334
relevancy Score: 0.8333333333333334


# DeepEval

In [ ]:
!pip install -U deepeval

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 555.6/555.6 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.5/118.5 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.2/196.2 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.1/89.1 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.3/289.3 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.6/40.6 kB 2.6 MB/s eta 0:00:00
  Attempting uninstall: click
    Found existing installation: click 8.2.1
    Uninstalling click-8.2.1:
      Successfully uninstall

[DeepEval Trial](https://docs.llamaindex.ai/en/stable/examples/evaluation/faithfulness_eval/)

In [ ]:
from deepeval.metrics import AnswerRelevancyMetric
from deepeval.test_case import LLMTestCase

retrieval_context = [node.get_content() for node in response.source_nodes]

test_case = LLMTestCase(
    input=prompt,
    actual_output=response.response,
    retrieval_context=retrieval_context
)

answer_relevancy_metric = AnswerRelevancyMetric()

# Evaluate
answer_relevancy_metric.measure(test_case)
print(answer_relevancy_metric.score)
print(answer_relevancy_metric.reason)

In [ ]:
from deepeval.integrations.llama_index import DeepEvalFaithfulnessEvaluator

evaluator = DeepEvalAnswerRelevancyEvaluator()
evaluation_result = evaluator.evaluate_response(
    query=query_engine, response=response
)
print(evaluation_result)

# Implement Rerank
[Query Bundle](https://docs.llamaindex.ai/en/v0.10.17/api/llama_index.core.schema.QueryBundle.html#querybundle)

In [ ]:
#from llama_index.core.retrievers import VectorIndexRetriever (Already Imported)
from llama_index.core import QueryBundle
from llama_index.core.indices.postprocessor import LLMRerank

#from typing import List

retriever2 = VectorIndexRetriever(
        index=index,
        similarity_top_k=5,
        #vector_store_query_mode="default",

    )

query_bundle = QueryBundle(prompt)

retrieved_nodes = retriever2.retrieve(query_bundle)

reranker = LLMRerank(
            choice_batch_size=5,
            top_n=5,
        )
retrieved_nodes = reranker.postprocess_nodes(
            retrieved_nodes, query_bundle
        )

#pprint_response(retrieved_nodes,show_source=True)

In [ ]:
query_engine2=RetrieverQueryEngine(retriever=retriever2)

In [ ]:
print(f"Retrieved {len(retrieved_nodes)} nodes:")
for i, node in enumerate(retrieved_nodes):
    print(f"\n--- Node {i + 1} ---")
    print(node.get_content())
    print(f"Score: {node.score}")

In [ ]:
response2=query_engine2.query(query_bundle)

In [ ]:
from llama_index.core.response.pprint_utils import pprint_response
pprint_response(response2,show_source=True)
#print(response)